In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from nbainjuries import injury
from datetime import datetime, timedelta
import warnings
import duckdb
import os
import requests

cwd = os.path.abspath(os.getcwd()).replace("\\", "/")
if cwd.startswith("C:/Users/Rodolfo/"):
    RUN_LOCATION = "local"
else:
    RUN_LOCATION = "cloud"
time_offset = {"local": 3, "cloud": -5}

print("Current working dir:", cwd)
print("RUN_LOCATION =", RUN_LOCATION)

Current working dir: C:/Users/Rodolfo/Jupyter_files/FantasyBasketball/notebooks
RUN_LOCATION = local


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings("ignore")

year = 2025
now = str((datetime.now() + timedelta(hours=time_offset[RUN_LOCATION]) + timedelta(hours=-3)).date())
print(f"Today's date:", now)

con = duckdb.connect(database=":memory:")
categories = ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK']

folders = os.listdir('../tables/')
df = pd.DataFrame()
for yr in folders:
    df_temp = pd.read_csv(f"../tables/{yr}/season_gamelogs.csv")
    df_temp.insert(0, 'Season', int(yr))
    df = pd.concat([df, df_temp])
df['Date'] = pd.to_datetime(df.Date)
df = df.rename(columns={"TRB": "REB", "3PM": "TPM", "3PA": "TPA"})
df['STL_BLK'] = df.STL + df.BLK
df['PR'] = df.PTS + df.REB 
df['PA'] = df.PTS + df.AST
df['RA'] = df.REB + df.AST
df['PRA'] = df.PTS + df.REB + df.AST

# # CONTROL DATE TO GO BACK AND RELOAD HISTORICAL DATA
# df = df[(df.Date != now)]

df

Today's date: 2025-11-25


,Season,game_id,Date,Team,Team_type,Opp,Opp_type,Player,Pos,Active,MP,FG,FGA,FG%,TPM,TPA,3P%,FT,FTA,FT%,ORB,DRB,REB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,DD,TD,Fpts,STL_BLK,PR,PA,RA,PRA
0,2023,20231024_LAL_DEN,2023-10-24,DEN,Home,LAL,Away,Christian Braun,SG,1,19.33,2,5,0.400,0,1,0.000,1,2,0.5,1,2,3,2,0,1,1,1,5,3.9,5,0,0,8.5,1,8,7,5,10
1,2023,20231024_LAL_DEN,2023-10-24,DEN,Home,LAL,Away,Kentavious Caldwell-Pope,SG,1,36.25,8,12,0.667,2,3,0.667,2,2,1.0,1,1,2,1,3,1,3,5,20,15.2,10,0,0,19.0,4,22,21,3,23
2,2023,20231024_LAL_DEN,2023-10-24,DEN,Home,LAL,Away,Collin Gillespie,PG,1,0.72,0,0,0.000,0,0,0.000,0,0,0.0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0
3,2023,20231024_LAL_DEN,2023-10-24,DEN,Home,LAL,Away,Aaron Gordon,PF,1,34.98,7,11,0.636,1,2,0.500,0,0,0.0,2,5,7,5,2,1,0,0,15,19.2,6,0,0,26.0,3,22,20,12,27
4,2023,20231024_LAL_DEN,2023-10-24,DEN,Home,LAL,Away,Justin Holiday,SG,0,0.00,0,0,0.000,0,0,0.000,0,0,0.0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6832,2025,20251124_MIN_SAC,2025-11-24,SAC,Home,MIN,Away,Maxime Raynaud,C,1,13.92,0,4,0.000,0,1,0.000,1,2,0.5,1,1,2,0,0,0,0,3,1,-2.4,-5,0,0,2.5,0,3,1,2,3
6833,2025,20251124_MIN_SAC,2025-11-24,SAC,Home,MIN,Away,Dario Saric,C,0,0.00,0,0,0.000,0,0,0.000,0,0,0.0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0
6834,2025,20251124_MIN_SAC,2025-11-24,SAC,Home,MIN,Away,Doug McDermott,SF,0,0.00,0,0,0.000,0,0,0.000,0,0,0.0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0
6835,2025,20251124_MIN_SAC,2025-11-24,SAC,Home,MIN,Away,Keegan Murray,PF,1,44.40,10,19,0.526,3,5,0.600,3,3,1.0,2,12,14,2,1,1,0,3,26,23.6,6,1,0,35.5,2,40,28,16,42


In [3]:
%run ./common_utils.ipynb

In [4]:
df_gms = pd.read_csv(f"../tables/{year}/nba_schedule.csv")
df_gms = df_gms[(df_gms.Date == now)]
tms_today = df_gms.AwayABV.tolist() + df_gms.HomeABV.tolist()
display(df_gms)

,Date,StartTime_ET,AwayTeam,AwayABV,HomeTeam,HomeABV,Arena,AwayB2B,HomeB2B,rtrvd
260,2025-11-25,7:00p,Atlanta Hawks,ATL,Washington Wizards,WAS,Capital One Arena,0,0,0
261,2025-11-25,8:00p,Orlando Magic,ORL,Philadelphia 76ers,PHI,Xfinity Mobile Arena,0,0,0
262,2025-11-25,11:00p,Los Angeles Clippers,LAC,Los Angeles Lakers,LAL,Crypto.com Arena,0,0,0


In [5]:
df_teams = pd.read_csv("../src/team_info_xref.csv")

df_inj = injury.get_reportdata(datetime.now() + timedelta(hours=time_offset[RUN_LOCATION]), return_df=True)
df_inj = df_inj.rename(columns={"Game Date": "Date", "Player Name": "Player", "Current Status": "Status"})
df_inj['Player'] = df_inj.Player.str.split(",").str[1] + " " + df_inj.Player.str.split(",").str[0]
df_inj['Player'] = df_inj['Player'].str.strip()
df_inj['Date'] = pd.to_datetime(df_inj['Date'])
df_inj = con.execute(f"""SELECT Date, ABV as Team, Player, Status, Reason FROM df_inj 
                         JOIN df_teams ON df_inj.Team = df_teams.Team
                         WHERE Date = '{now}' AND Status = 'Out'""").fetchdf()

df_best_out = con.execute(f"""SELECT *, RANK() OVER (PARTITION BY Pos ORDER BY Fpts DESC) as Off_Rk FROM
                              (SELECT Team, Player, Pos, AVG(Fpts) as Fpts FROM df 
                              WHERE Season = 2025 AND Fpts > 0 AND Pos != 'None'
                              GROUP BY Team, Player, Pos)""").fetchdf()
df_best_out = con.execute(f"""SELECT * EXCLUDE(t2.Team, t2.Player) FROM df_best_out t1 
                              JOIN df_inj t2 ON t1.Team = t2.Team AND t1.Player = t2.Player
                              ORDER BY Fpts DESC""").fetchdf()
display(df_best_out)

Validated Injury-Report_2025-11-25_07PM.


,Team,Player,Pos,Fpts,Off_Rk,Date,Status,Reason
0,ORL,Paolo Banchero,PF,25.125000,8,2025-11-25,Out,Injury/Illness - Left Groin; Strain
1,PHI,Joel Embiid,C,21.416667,17,2025-11-25,Out,Injury/Illness - Right Knee; Injury Management
2,WAS,Kyshawn George,SF,21.366667,14,2025-11-25,Out,Injury/Illness - N/a; Illness
3,PHI,VJ Edgecombe,SG,20.666667,10,2025-11-25,Out,Injury/Illness - Left Calf; Tightness
4,ATL,Trae Young,PG,19.600000,19,2025-11-25,Out,Injury/Illness - Right Knee; MCL Sprain
5,PHI,Kelly Oubre Jr.,PF,19.318182,17,2025-11-25,Out,Injury/Illness - Left Knee; Sprain
6,LAL,Deandre Ayton,C,18.300000,25,2025-11-25,Out,Injury/Illness - Right Knee; Contusion
7,PHI,Paul George,PF,17.833333,21,2025-11-25,Out,Injury/Illness - Right Ankle; Sprain
8,WAS,Tre Johnson,SG,11.233333,53,2025-11-25,Out,Injury/Illness - Left Hip Flexor; Strain
9,WAS,Marvin Bagley III,PF,11.142857,45,2025-11-25,Out,Injury/Illness - Right Hip; Contusion


In [6]:
def get_sportsbook():
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Accept": "*/*",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://sportsbook.draftkings.com/",
        "Origin": "https://sportsbook.draftkings.com"
    }

    dk_cats = {"PTS": 12488, "AST": 12495, "REB": 12492, "STL": 13508, "BLK": 13780, "STL_BLK": 13781, "TPM": 12497, 
               "PA": 9973, "PR": 9976, "RA": 9974, "PRA": 5001}
    df_lines = pd.DataFrame()
    for key, value in dk_cats.items():
        response = requests.get(f"https://sportsbook-nash.draftkings.com/sites/US-SB/api/sportscontent/controldata/league/leagueSubcategory/v1/markets?isBatchable=false&templateVars=42648%2C{value}&eventsQuery=%24filter%3DleagueId%20eq%20%2742648%27%20AND%20clientMetadata%2FSubcategories%2Fany%28s%3A%20s%2FId%20eq%20%27{value}%27%29&marketsQuery=%24filter%3DclientMetadata%2FsubCategoryId%20eq%20%27{value}%27%20AND%20tags%2Fall%28t%3A%20t%20ne%20%27SportcastBetBuilder%27%29&include=Events&entity=events", headers=headers)
        if response.status_code != 200:
            raise Exception('Bad Request')

        plyr_names = []
        pnt_lines = []
        for i in response.json()['selections']:
            plyr_names.append(i['participants'][0]['name'])
            pnt_lines.append(i['points'])
        df_dk = pd.DataFrame({"Player": plyr_names, f"{key}_line": pnt_lines}).drop_duplicates().reset_index(drop=True)

        if df_lines.empty:
            df_lines = df_dk
        else:
            df_lines = pd.merge(df_lines, df_dk, on="Player", how="outer")

    df_lines = pd.merge(pd.read_csv(f"../tables/{year}/plyr_pos_xref.csv"), df_lines, on='Player', how='right')
    df_lines.insert(0, 'Date', pd.to_datetime(now))

    partition_save_df(df_lines, f"../tables/{year}/parlay_lines.csv") 
    display(df_lines)

    return df_lines

df_lines = get_sportsbook()

../tables/2025/parlay_lines.csv saved!


,Date,Team,Player,Pos,PTS_line,AST_line,REB_line,STL_line,BLK_line,STL_BLK_line,TPM_line,PA_line,PR_line,RA_line,PRA_line
0,2025-11-25,PHI,Andre Drummond,C,11.5,NaN,12.5,NaN,NaN,NaN,NaN,NaN,24.5,NaN,25.5
1,2025-11-25,ORL,Anthony Black,PG,12.5,3.5,3.5,NaN,NaN,NaN,NaN,15.5,15.5,6.5,19.5
2,2025-11-25,LAL,Austin Reaves,SG,22.5,4.5,4.5,0.5,NaN,NaN,2.5,27.5,27.5,9.5,32.5
3,2025-11-25,LAC,Bogdan Bogdanovic,SG,8.5,NaN,NaN,NaN,NaN,NaN,1.5,NaN,NaN,NaN,NaN
4,2025-11-25,LAC,Brook Lopez,C,5.5,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN
5,2025-11-25,ORL,Desmond Bane,SG,20.5,4.5,4.5,0.5,NaN,NaN,2.5,25.5,25.5,9.5,30.5
6,2025-11-25,ORL,Franz Wagner,SF,25.5,4.5,6.5,1.5,0.5,1.5,1.5,30.5,32.5,10.5,36.5
7,2025-11-25,LAL,Gabe Vincent,PG,4.5,NaN,NaN,NaN,NaN,NaN,1.5,NaN,NaN,NaN,NaN
8,2025-11-25,ORL,Goga Bitadze,C,5.5,NaN,4.5,NaN,NaN,NaN,NaN,NaN,10.5,NaN,NaN
9,2025-11-25,LAC,Ivica Zubac,C,17.5,2.5,11.5,0.5,0.5,1.5,NaN,19.5,28.5,13.5,31.5


In [18]:
def pick_finder(stat, collect=False):
    df_mtch = df_gms[['AwayABV', 'HomeABV', 'AwayB2B', 'HomeB2B']]
    df_mtch = df_mtch.rename(columns={"AwayABV": "Team", "HomeABV": "Opp", "AwayB2B": "B2B"})[['Team', 'Opp', 'B2B']]
    df_mtch2 = df_mtch.copy().rename(columns={"Team": "Opp", "Opp": "Team", "HomeB2B": "B2B"})[['Team', 'Opp', 'B2B']]
    df_mtch = pd.concat([df_mtch, df_mtch2])
    
    df_rk = con.execute(f"""SELECT *, RANK() OVER (PARTITION BY Pos ORDER BY Off_{stat} DESC) as Off_Rk FROM
                            (SELECT Team, Player, Pos, AVG({stat}) as Off_{stat} FROM df 
                            WHERE Season = 2025 AND {stat} > 0 AND Pos != 'None'
                            AND Team IN ({str(tms_today).replace("[", "").replace("]", "")})
                            GROUP BY Team, Player, Pos)""").fetchdf()
    df_rk_l5 = con.execute(f"""WITH last5 AS (
                                    SELECT Team, Player, Pos, {stat} AS stat_val,
                                    ROW_NUMBER() OVER (PARTITION BY Team, Player ORDER BY Date DESC) AS rn
                                    FROM df
                                    WHERE Season = 2025 AND {stat} > 0 AND Pos != 'None'
                                    AND Team IN ({str(tms_today).replace('[','').replace(']','')})
                            )
                            SELECT Team, Player, Pos, AVG(stat_val) AS Off_L5_{stat},
                            FROM last5
                            WHERE rn <= 5
                            GROUP BY Team, Player, Pos
                            """).fetchdf()

    df_rk = con.execute(f"""SELECT df_rk.Team, df_rk.Player, df_rk.Pos, Off_{stat}, Off_L5_{stat}, Off_Rk FROM df_rk JOIN df_rk_l5 
                            ON df_rk.Team = df_rk_l5.Team AND df_rk.Player = df_rk_l5.Player""").fetchdf()
    df_save = pd.DataFrame()
    for pos in ['PG', 'SG', 'SF', 'PF', 'C']:
        print(pos)
        df_def = con.execute(f"""SELECT Team, '{pos}' AS Pos, Def_{stat}, Def_Rk FROM
                                 (SELECT *, RANK() OVER (ORDER BY Def_{stat}) as Def_Rk FROM
                                 (SELECT Opp as Team, AVG({stat}) as Def_{stat} FROM df 
                                 WHERE MP >= 27 AND Season = 2025 AND Pos = '{pos}'
                                 GROUP BY Opp
                                 ORDER BY AVG(Fpts) DESC))
                                 WHERE Team IN ({str(tms_today).replace("[", "").replace("]", "")})""").fetchdf()

        df_def_l5 = con.execute(f"""WITH last5 AS (
                                    SELECT Opp AS Team, {stat} AS stat_val, 
                                    ROW_NUMBER() OVER (PARTITION BY Opp ORDER BY Date DESC) AS rn
                                    FROM df
                                    WHERE MP >= 27 AND Season = 2025 AND Pos = '{pos}'
                                    )

                                    SELECT Team, '{pos}' AS Pos, AVG(stat_val) AS Def_L5_{stat} FROM last5
                                    WHERE rn <= 5
                                    GROUP BY Team
                                    HAVING Team IN ({str(tms_today).replace('[','').replace(']','')})
                                    """).fetchdf()
        df_def = con.execute(f"""SELECT df_def.Team, df_def.Pos, Def_{stat}, Def_L5_{stat}, Def_Rk FROM df_def 
                               JOIN df_def_l5 ON df_def.Team = df_def_l5.Team""").fetchdf()
        
        # Piece together the current matchups with offensive rankings vs defensive rankings
        df_picks = con.execute(f"""SELECT df_mtch.*, df_rk.* EXCLUDE(Team) FROM df_mtch 
                                  JOIN df_rk ON df_mtch.Team = df_rk.Team
                                  WHERE Pos = '{pos}'""").fetchdf()
        df_picks = con.execute(f"""SELECT df_picks.* EXCLUDE(Opp, Pos), Opp, df_def.Def_{stat}, df_def.Def_L5_{stat}, 
                                   df_def.Def_Rk FROM df_picks 
                                   JOIN df_def ON df_picks.Opp = df_def.Team""").fetchdf()
        df_picks['Rk_Diff'] = df_picks['Def_Rk'] - df_picks['Off_Rk']
        df_picks = con.execute(f"""SELECT t1.* EXCLUDE(Rk_Diff) FROM df_picks t1
                                LEFT JOIN df_inj t2 ON t1.Team = t2.Team AND t1.Player = t2.Player
                                WHERE Status IS NULL
                                ORDER BY Rk_Diff DESC""").fetchdf()
        
        # head to head
        df_h2h = pd.DataFrame()
        for index, row in df_picks.iterrows():
            query = f"""SELECT * EXCLUDE(Gms) FROM
                       (SELECT Player, COUNT(*) as Gms, AVG({stat}) AS AVG_{stat}_H2H FROM df 
                       WHERE Player = ? AND Opp = ? AND Date >= '2023-10-23'
                       GROUP BY Player)
                       WHERE Gms >= 4"""
            df_temp = con.execute(query, [row["Player"], row["Opp"]]).fetchdf()
            df_h2h = pd.concat([df_h2h, df_temp])
        try:
            df_picks = con.execute(f"""SELECT Team, B2B, t1.Player, Off_{stat}, Off_L5_{stat}, t2.AVG_{stat}_H2H, Off_Rk, 
                                   t1.* EXCLUDE(Team, B2B, Player, Off_{stat}, Off_L5_{stat}, Off_Rk)
                                   FROM df_picks t1 LEFT JOIN df_h2h t2 
                                   ON t1.Player = t2.Player""").fetchdf()
        except:
            pass
        df_picks = con.execute(f"""SELECT df_picks.*, df_lines.{stat}_line FROM df_picks LEFT JOIN df_lines 
                                   ON df_picks.Team = df_lines.Team AND df_picks.Player = df_lines.Player
                                   ORDER BY Def_Rk DESC""").fetchdf()
        df_save = pd.concat([df_save, df_picks], ignore_index=True)
        if collect == False:
            display(df_picks)
    if collect == True:
        df_save.insert(0, 'Date', pd.to_datetime(now))
        return df_save

In [10]:
for i in categories:
    print(f"==={i}===")
    pick_finder(i)

===PTS===
PG


,Team,B2B,Player,Off_PTS,Off_L5_PTS,AVG_PTS_H2H,Off_Rk,Opp,Def_PTS,Def_L5_PTS,Def_Rk,PTS_line
0,PHI,0,Tyrese Maxey,33.000000,35.0,26.714286,2,ORL,23.375000,26.6,29,32.5
1,PHI,0,Kyle Lowry,3.000000,3.0,2.666667,13,ORL,23.375000,26.6,29,NaN
2,ATL,0,Vit Krejci,11.928571,8.6,NaN,6,WAS,22.833333,20.6,25,NaN
3,LAL,0,Gabe Vincent,4.000000,4.0,8.000000,11,LAC,20.466667,21.0,16,4.5
4,LAL,0,Luka Doncic,34.500000,30.8,31.000000,1,LAC,20.466667,21.0,16,32.5
5,WAS,0,Bub Carrington,6.733333,6.0,14.750000,9,ATL,19.538462,19.2,9,NaN
6,ORL,0,Anthony Black,11.833333,15.2,6.875000,7,PHI,18.357143,18.6,6,12.5
7,ORL,0,Jalen Suggs,14.384615,18.0,18.000000,5,PHI,18.357143,18.6,6,15.5
8,ORL,0,Tyus Jones,4.428571,5.0,11.000000,10,PHI,18.357143,18.6,6,NaN
9,LAC,0,James Harden,27.812500,34.0,17.142857,3,LAL,17.928571,24.0,5,24.5


SG


,Team,B2B,Player,Off_PTS,Off_L5_PTS,AVG_PTS_H2H,Off_Rk,Opp,Def_PTS,Def_L5_PTS,Def_Rk,PTS_line
0,LAC,0,Bogdan Bogdanovic,10.375000,11.600000,13.000000,8,LAL,18.200000,16.6,24,8.5
1,LAC,0,Bradley Beal,8.166667,8.800000,19.666667,14,LAL,18.200000,16.6,24,NaN
2,LAC,0,Cam Christie,5.727273,4.200000,NaN,17,LAL,18.200000,16.6,24,NaN
3,ORL,0,Desmond Bane,17.444444,21.800000,NaN,3,PHI,17.600000,15.8,19,20.5
4,ORL,0,Jase Richardson,8.000000,8.000000,NaN,15,PHI,17.600000,15.8,19,NaN
5,LAL,0,Austin Reaves,27.615385,23.400000,12.571429,1,LAC,17.000000,21.4,14,22.5
6,LAL,0,Marcus Smart,9.461538,9.200000,NaN,11,LAC,17.000000,21.4,14,5.5
7,LAL,0,Bronny James,4.200000,4.200000,NaN,19,LAC,17.000000,21.4,14,NaN
8,PHI,0,Quentin Grimes,16.750000,15.600000,5.833333,4,ORL,16.866667,12.0,12,17.5
9,PHI,0,Jared McCain,9.333333,9.333333,NaN,12,ORL,16.866667,12.0,12,12.5


SF


...

,Team,B2B,Player,Off_STL_BLK,Off_L5_STL_BLK,AVG_STL_BLK_H2H,Off_Rk,Opp,Def_STL_BLK,Def_L5_STL_BLK,Def_Rk,STL_BLK_line
0,ATL,0,Jalen Johnson,2.294118,1.80,2.000000,4,WAS,2.181818,1.2,27,NaN
1,ATL,0,Zaccharie Risacher,2.000000,2.00,1.750000,6,WAS,2.181818,1.2,27,NaN
2,PHI,0,Justin Edwards,1.833333,1.80,NaN,10,ORL,1.500000,1.4,12,NaN
3,WAS,0,Corey Kispert,1.000000,1.00,0.250000,16,ATL,1.444444,0.8,10,NaN
4,WAS,0,Justin Champagnie,1.800000,1.80,NaN,11,ATL,1.444444,0.8,10,NaN
5,WAS,0,Khris Middleton,1.625000,1.60,NaN,13,ATL,1.444444,0.8,10,NaN
6,WAS,0,Cam Whitmore,1.500000,1.60,NaN,14,ATL,1.444444,0.8,10,NaN
7,WAS,0,Will Riley,1.000000,1.00,NaN,16,ATL,1.444444,0.8,10,NaN
8,LAL,0,LeBron James,1.000000,1.00,2.428571,16,LAC,1.388889,1.2,9,1.5
9,LAL,0,Dalton Knecht,2.166667,2.40,0.750000,5,LAC,1.388889,1.2,9,NaN


PF


,Team,B2B,Player,Off_STL_BLK,Off_L5_STL_BLK,AVG_STL_BLK_H2H,Off_Rk,Opp,Def_STL_BLK,Def_L5_STL_BLK,Def_Rk,STL_BLK_line
0,ATL,0,Mouhamed Gueye,1.666667,1.600000,0.000000,10,WAS,2.538462,2.2,29,NaN
1,ATL,0,Asa Newell,2.000000,2.000000,NaN,5,WAS,2.538462,2.2,29,NaN
2,LAL,0,Jake LaRavia,2.250000,2.000000,1.750000,2,LAC,2.000000,2.8,18,NaN
3,LAL,0,Rui Hachimura,1.444444,1.200000,0.750000,12,LAC,2.000000,2.8,18,NaN
4,LAL,0,Maxi Kleber,1.333333,1.333333,0.625000,13,LAC,2.000000,2.8,18,NaN
5,LAL,0,Jarred Vanderbilt,1.900000,1.600000,1.800000,8,LAC,2.000000,2.8,18,NaN
6,PHI,0,Trendon Watford,1.285714,1.200000,0.428571,14,ORL,1.583333,1.6,9,NaN
7,LAC,0,John Collins,1.846154,1.800000,2.142857,9,LAL,1.500000,2.0,6,NaN
8,LAC,0,Nicolas Batum,2.090909,1.800000,1.166667,4,LAL,1.500000,2.0,6,NaN


C


,Team,B2B,Player,Off_STL_BLK,Off_L5_STL_BLK,AVG_STL_BLK_H2H,Off_Rk,Opp,Def_STL_BLK,Def_L5_STL_BLK,Def_Rk,STL_BLK_line
0,ORL,0,Goga Bitadze,2.066667,2.0,2.625000,6,PHI,2.545455,1.0,22,NaN
1,ORL,0,Wendell Carter Jr.,2.000000,2.2,1.142857,7,PHI,2.545455,1.0,22,NaN
2,LAL,0,Jaxson Hayes,1.428571,1.4,1.125000,12,LAC,2.300000,2.0,17,NaN
3,LAC,0,Brook Lopez,1.583333,1.4,2.250000,11,LAL,2.250000,2.2,15,NaN
4,LAC,0,Ivica Zubac,1.769231,1.6,1.333333,10,LAL,2.250000,2.2,15,1.5
5,WAS,0,Alex Sarr,3.363636,3.4,3.750000,1,ATL,2.250000,3.6,15,NaN
6,LAC,0,Yanic Konan Niederhauser,1.000000,1.0,NaN,13,LAL,2.250000,2.2,15,NaN
7,ATL,0,Onyeka Okongwu,3.000000,3.6,1.857143,2,WAS,1.888889,1.6,9,NaN
8,ATL,0,Kristaps Porzingis,2.400000,2.8,0.400000,3,WAS,1.888889,1.6,9,NaN
9,PHI,0,Andre Drummond,1.857143,1.8,2.142857,9,ORL,1.600000,1.0,4,NaN


In [19]:
df_stats = pd.DataFrame()
for i in categories:
    print(f"==={i}===")
    df_temp = pick_finder(i, collect=True)
    df_stats = pd.concat([df_stats, df_temp], ignore_index=True)
    
df_stats = con.execute("SELECT Date, Team, B2B, Player, Opp, * EXCLUDE(Date, Team, B2B, Player, Opp, Off_Rk, Def_Rk) FROM df_stats").fetchdf()
df_save = pd.DataFrame()
for player in df_stats.Player.unique():
    df_temp = df_stats[(df_stats.Player == player)]
    for col in df_temp.columns.difference(['Date', 'Team', 'Player']):
        df_temp[col] = df_temp[col].ffill()
        df_temp[col] = df_temp[col].backfill()
    df_temp = df_temp.drop_duplicates()
    df_save = pd.concat([df_save, df_temp])
print('Saving for Date:', now)
partition_save_df(df_save, f"../tables/{year}/parlay_stats.csv")
display(df_save)

===PTS===
PG
SG
SF
PF
C
===AST===
PG
SG
SF
PF
C
===REB===
PG
SG
SF
PF
C
===PR===
PG
SG
SF
PF
C
===PA===
PG
SG
SF
PF
C
===RA===
PG
SG
SF
PF
C
===PRA===
PG
SG
SF
PF
C
===TPM===
PG
SG
SF
PF
C
===STL===
PG
SG
SF
PF
C
===BLK===
PG
SG
SF
PF
C
===STL_BLK===
PG
SG
SF
PF
C
Saving for Date: 2025-11-25
../tables/2025/parlay_stats.csv saved!


,Date,Team,B2B,Player,Opp,Off_PTS,Off_L5_PTS,AVG_PTS_H2H,Def_PTS,Def_L5_PTS,PTS_line,Off_AST,Off_L5_AST,AVG_AST_H2H,Def_AST,Def_L5_AST,AST_line,Off_REB,Off_L5_REB,AVG_REB_H2H,Def_REB,Def_L5_REB,REB_line,Off_PR,Off_L5_PR,AVG_PR_H2H,Def_PR,Def_L5_PR,PR_line,Off_PA,Off_L5_PA,AVG_PA_H2H,Def_PA,Def_L5_PA,PA_line,Off_RA,Off_L5_RA,AVG_RA_H2H,Def_RA,Def_L5_RA,RA_line,Off_PRA,Off_L5_PRA,AVG_PRA_H2H,Def_PRA,Def_L5_PRA,PRA_line,Off_TPM,Off_L5_TPM,AVG_TPM_H2H,Def_TPM,Def_L5_TPM,TPM_line,Off_STL,Off_L5_STL,AVG_STL_H2H,Def_STL,Def_L5_STL,STL_line,Off_BLK,Off_L5_BLK,AVG_BLK_H2H,Def_BLK,Def_L5_BLK,BLK_line,Off_STL_BLK,Off_L5_STL_BLK,AVG_STL_BLK_H2H,Def_STL_BLK,Def_L5_STL_BLK,STL_BLK_line
0,2025-11-25,PHI,0,Tyrese Maxey,ORL,33.000000,35.0,26.714286,23.375000,26.6,32.5,7.750000,6.6,4.428571,7.500000,7.6,6.5,4.625000,4.000000,3.571429,4.812500,4.6,3.5,37.625000,39.0,30.285714,28.187500,31.2,36.5,40.750000,41.6,31.142857,30.875000,34.2,39.5,12.375000,10.6,8.000000,12.312500,12.2,10.5,45.375000,45.6,34.714286,35.687500,38.8,43.5,4.125000,4.40,2.857143,2.375000,3.6,3.5,1.733333,2.60,1.714286,1.750000,1.6,1.5,1.750000,2.2,0.428571,0.312500,0.0,NaN,2.500000,3.4,2.142857,2.062500,1.6,NaN
1,2025-11-25,PHI,0,Kyle Lowry,ORL,3.000000,3.0,2.666667,23.375000,26.6,NaN,1.000000,1.0,2.166667,7.500000,7.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,3.0,3.500000,28.187500,31.2,NaN,4.000000,4.0,4.833333,30.875000,34.2,NaN,1.000000,1.0,3.000000,12.312500,12.2,NaN,4.000000,4.0,5.666667,35.687500,38.8,NaN,1.000000,1.00,0.333333,2.375000,3.6,NaN,1.000000,1.00,0.833333,1.750000,1.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.0,1.000000,2.062500,1.6,NaN
2,2025-11-25,ATL,0,Vit Krejci,WAS,11.928571,8.6,NaN,22.833333,20.6,NaN,2.000000,2.4,NaN,8.166667,8.2,NaN,2.285714,2.200000,NaN,5.416667,6.8,NaN,13.266667,10.6,NaN,28.250000,27.4,NaN,13.000000,10.4,NaN,31.000000,28.8,NaN,4.000000,3.8,NaN,13.583333,15.0,NaN,15.133333,12.4,NaN,36.416667,35.6,NaN,3.307692,3.60,NaN,2.250000,1.6,NaN,2.250000,2.25,NaN,1.750000,2.4,NaN,1.000000,1.0,NaN,0.583333,0.8,NaN,1.714286,1.8,NaN,2.333333,3.2,NaN
3,2025-11-25,LAL,0,Gabe Vincent,LAC,4.000000,4.0,8.000000,20.466667,21.0,4.5,2.000000,2.0,2.000000,5.533333,4.0,NaN,1.333333,1.333333,1.750000,3.866667,3.2,NaN,4.800000,4.8,9.750000,24.333333,24.2,NaN,5.600000,5.6,10.000000,26.000000,25.0,NaN,2.400000,2.4,3.750000,9.400000,7.2,NaN,6.400000,6.4,11.750000,29.866667,28.2,NaN,1.250000,1.25,2.000000,2.800000,2.6,1.5,1.000000,1.00,1.250000,1.466667,1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.0,1.750000,1.866667,1.2,NaN
4,2025-11-25,LAL,0,Luka Doncic,LAC,34.500000,30.8,31.000000,20.466667,21.0,32.5,8.916667,8.6,8.272727,5.533333,4.0,7.5,8.750000,7.800000,7.181818,3.866667,3.2,9.5,43.250000,38.6,38.181818,24.333333,24.2,42.5,43.416667,39.4,39.272727,26.000000,25.0,40.5,17.666667,16.4,15.454545,9.400000,7.2,17.5,52.166667,47.2,46.454545,29.866667,28.2,50.5,3.416667,2.80,3.000000,2.800000,2.6,3.5,2.400000,2.40,1.363636,1.466667,1.2,1.5,1.500000,1.5,0.454545,0.400000,0.0,0.5,3.000000,3.0,1.818182,1.866667,1.2,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,2025-11-25,PHI,0,Andre Drummond,ORL,9.214286,10.6,6.428571,14.000000,16.0,11.5,1.375000,1.4,0.285714,2.133333,2.4,NaN,10.600000,14.600000,8.142857,9.600000,11.6,12.5,19.200000,25.2,14.571429,23.600000,27.6,24.5,10.000000,11.4,6.714286,16.133333,18.4,NaN,11.333333,15.4,8.428571,11.733333,14.0,NaN,19.933333,26.0,14.857143,25.733333,30.0,25.5,1.333333,1.40,0.000000,0.466667,0.6,NaN,1.200000,1.20,1.714286,0.400000,0.4,NaN,1.272727,1.2,0.428571,1.200000,0.6,NaN,1.857143,1.8,2.142857,1.600000,1.0,NaN
65,2025-11-25,WAS,0,Alex Sarr,ATL,18.142857,16.4,9.500000,13.666667,14.4,NaN,3.642857,3.2,1.250000,3.416667,4.6,NaN,8.357143,9.200000,5.250000,9.583333,8.2,NaN,26.500000,25.6,14.750000